In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading data
raw_data = pd.read_csv('../data/train.csv')

In [3]:
#about the data
raw_data.head()
raw_data['Track Name'].nunique()

15129

## Data Imputing

In [4]:
raw_data.isnull().sum()

Artist Name              0
Track Name               0
Popularity             428
danceability             0
energy                   0
key                   2014
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness      4377
liveness                 0
valence                  0
tempo                    0
duration_in min/ms       0
time_signature           0
Class                    0
dtype: int64

In [5]:
# filling missing value with zeros. 
df = raw_data.fillna(0)
# didnt work greatly

In [6]:
# drop all na values
#df = raw_data.dropna()
# didnt work well

In [7]:
# impute using the mean of each column
"""df = raw_data
df['Popularity'].fillna(raw_data['Popularity'].mean(), inplace=True) # imputing the popularity column with the mean
df['key'].fillna(raw_data['key'].mean(), inplace=True) # imputing the key with the mean 
df['instrumentalness'].fillna(raw_data['instrumentalness'].mean(), inplace=True) # imputing the instrumentalness with mean
"""
df.isnull().sum()

Artist Name           0
Track Name            0
Popularity            0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_in min/ms    0
time_signature        0
Class                 0
dtype: int64

In [8]:
# artist encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['Artist Name'] = encoder.fit_transform(df['Artist Name'])
#df.drop('Artist Name', inplace=True, axis=1)
df.describe()

,Artist Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
count,17996.000000,17996.000000,17996.000000,17996.000000,17996.000000,17996.000000,17996.000000,17996.000000,17996.000000,17996.000000,17996.000000,17996.000000,17996.000000,1.799600e+04,17996.000000,17996.000000
mean,4580.629584,43.453490,0.543433,0.662777,5.286286,-7.910660,0.636753,0.079707,0.247082,0.134375,0.196170,0.486208,122.623294,2.007445e+05,3.924039,6.695821
std,2651.222266,18.506174,0.166268,0.235373,3.549335,4.049151,0.480949,0.083576,0.310632,0.275251,0.159212,0.240195,29.571527,1.119891e+05,0.361618,3.206073
min,0.000000,0.000000,0.059600,0.000020,0.000000,-39.952000,0.000000,0.022500,0.000000,0.000000,0.011900,0.018300,30.557000,5.016500e-01,1.000000,0.000000
25%,2279.000000,32.000000,0.432000,0.509000,2.000000,-9.538000,0.000000,0.034800,0.004300,0.000001,0.097500,0.297000,99.620750,1.663370e+05,4.000000,5.000000
50%,4584.000000,43.000000,0.545000,0.700000,5.000000,-7.016000,1.000000,0.047400,0.081400,0.000343,0.129000,0.481000,120.065500,2.091600e+05,4.000000,8.000000
75%,6925.000000,56.000000,0.659000,0.860000,8.000000,-5.189000,1.000000,0.083000,0.434000,0.056600,0.258000,0.672000,141.969250,2.524900e+05,4.000000,10.000000
max,9148.000000,100.000000,0.989000,1.000000,11.000000,1.355000,1.000000,0.955000,0.996000,0.996000,1.000000,0.986000,217.416000,1.477187e+06,5.000000,10.000000


In [9]:
# track encoding 
#df = df.drop('Track Name',axis=1)
df['Track Name'] = encoder.fit_transform(df['Track Name'])
df.head()

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
0,1182,11406,60.0,0.854,0.564,1.0,-4.964,1,0.0485,0.017100,0.000000,0.0849,0.8990,134.071,234596.0,4,5
1,1092,4988,54.0,0.382,0.814,3.0,-7.230,1,0.0406,0.001100,0.004010,0.1010,0.5690,116.454,251733.0,4,10
2,7899,8144,35.0,0.434,0.614,6.0,-8.334,1,0.0525,0.486000,0.000196,0.3940,0.7870,147.681,109667.0,4,6
3,1987,6753,66.0,0.853,0.597,10.0,-6.528,0,0.0555,0.021200,0.000000,0.1220,0.5690,107.033,173968.0,4,5
4,6283,8167,53.0,0.167,0.975,2.0,-4.279,1,0.2160,0.000169,0.016100,0.1720,0.0918,199.060,229960.0,4,10


Normalize the data

In [10]:
df['Popularity'] = df['Popularity']/df['Popularity'].max()
df['key'] = df['key']/df['key'].max()
df['tempo'] = df['tempo']/df['tempo'].max()
df['loudness'] = df['loudness'] / abs(df['loudness'].min())

In [11]:
X = df.drop('Class', axis=1)
y = df[['Class']]


In [12]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
#scalar = MinMaxScaler()
#X_train = scalar.fit_transform(X_train)
#X_test = scalar.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression 
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
logreg.score(X_train, y_train) , logreg.score(X_test, y_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=300)
#knn.fit(X_train, y_train)
hyper_grid = {'n_neighbors': range(1,101,4)}

kfold = KFold(n_splits=10, random_state=123, shuffle=True)

loss='roc_auc'

#grid_search = GridSearchCV(knn,hyper_grid,cv=kfold,scoring=loss)
results = cross_val_score(knn, X_train, y_train, cv=kfold)

results.max()

In [ ]:
from sklearn.svm import SVC 
svm = SVC()
svm.fit(X_train, y_train)

In [ ]:
svm.score(X_train,y_train), svm.score(X_test, y_test)

In [18]:
cbc = CatBoostClassifier()

parameters = {'depth': [4,5,6,7,8], 
'learning_rate': [0.01,0.02,0.03],
'iterations': [3,5,10] }

Grid_CBC = GridSearchCV(estimator=cbc, param_grid=parameters, cv=2, n_jobs=1)
Grid_CBC.fit(X_train,y_train)

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)



0:	learn: 2.3811760	total: 5.14ms	remaining: 10.3ms
1:	learn: 2.3674777	total: 12.1ms	remaining: 6.07ms
2:	learn: 2.3522673	total: 17.7ms	remaining: 0us
0:	learn: 2.3823358	total: 4.19ms	remaining: 8.39ms
1:	learn: 2.3667647	total: 9.03ms	remaining: 4.52ms
2:	learn: 2.3505911	total: 14.2ms	remaining: 0us
0:	learn: 2.3646198	total: 8.7ms	remaining: 17.4ms
1:	learn: 2.3380721	total: 12.9ms	remaining: 6.46ms
2:	learn: 2.3117564	total: 17.2ms	remaining: 0us
0:	learn: 2.3669282	total: 4.07ms	remaining: 8.14ms
1:	learn: 2.3366642	total: 9.98ms	remaining: 4.99ms
2:	learn: 2.3061981	total: 14.1ms	remaining: 0us
0:	learn: 2.3482304	total: 3.81ms	remaining: 7.62ms
1:	learn: 2.3096477	total: 11.8ms	remaining: 5.92ms
2:	learn: 2.2722892	total: 16.9ms	remaining: 0us
0:	learn: 2.3516752	total: 4.39ms	remaining: 8.78ms
1:	learn: 2.3075628	total: 9ms	remaining: 4.5ms
2:	learn: 2.2644901	total: 13.8ms	remaining: 0us
0:	learn: 2.3811760	total: 5.39ms	remaining: 21.6ms
1:	learn: 2.3674777	total: 10.1ms	r

In [ ]:
cbc = CatBoostClassifier(silent=True)
cbc.fit(X_train, y_train)
cbc.score(X_test, y_test)